In [2]:
#
import os
import copy
import pytz
import sys
import warnings
import matplotlib
from pathlib import Path
import dataclasses
from typing import Dict, Union

#
sys.path.append("./src/")
sys.path.append("./")
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

# FinRL
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

# FinRL-Meta
from stable_baselines3 import A2C
from stable_baselines3 import DDPG
from stable_baselines3 import PPO
from stable_baselines3 import SAC
from stable_baselines3 import TD3

In [3]:
def config():
    #
    warnings.filterwarnings("ignore", category=UserWarning)  # TODO: zipline problem
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=RuntimeWarning)

    #
    matplotlib.use("Agg")


config()

In [4]:
#
import numpy as np
import pandas as pd
import seaborn as sns
import tqdm

#
from common.utils import now_time
from configuration.settings import ProjectDir, ExperimentDir
from rl.data.CompanyInfo import CompanyInfo
from rl.experiments._1_same_bigger_data_fundamental.train import (
    Program,
    get_dataset,
    dataset_name,
    get_env_kwargs,
    CustomDRLAgent,
)
from rl.experiments._1_same_bigger_data_fundamental.StockTradingEnv import StockTradingEnv

In [5]:
#
program = Program(
    prj_dir=ProjectDir(root=Path("/Users/zlapik/my-drive-zlapik/0-todo/ai-investing")),
    exp_dir=ExperimentDir(Path(os.getcwd())),
    DEBUG=False,
)
program.dataset = get_dataset(
    pd.read_csv(program.exp_dir.out.datasets.joinpath(f"{dataset_name}.csv"), index_col=0), purpose="test"
)
program.exp_dir.check_and_create_dirs()

In [6]:
from rl.experiments._1_same_bigger_data_fundamental.train import base_cols, data_cols, ratios_cols

print(base_cols)
print(data_cols)
print(ratios_cols)

['date', 'tic']
['open', 'high', 'low', 'close', 'volume']
['operatingProfitMargin', 'netProfitMargin', 'returnOnAssets', 'returnOnEquity', 'currentRatio', 'quickRatio', 'cashRatio', 'inventoryTurnover', 'receivablesTurnover', 'payablesTurnover', 'debtRatio', 'debtEquityRatio', 'priceEarningsRatio', 'priceBookValueRatio', 'dividendYield']


In [7]:
@dataclasses.dataclass
class LearnedAlgorithm:
    algorithm: str
    filename: Path
    learned_algorithm: Union[A2C, PPO, DDPG, A2C, TD3]


def get_algorithm(filename: Path):
    if "a2c" in filename.as_posix():
        return LearnedAlgorithm(algorithm="a2c", filename=filename, learned_algorithm=A2C.load(filename))


learned_algorithms = [get_algorithm(filepath) for filepath in program.exp_dir.out.algorithms.glob("*")]

In [8]:
env_kwargs = get_env_kwargs(program.dataset)
env_gym = StockTradingEnv(df=program.dataset, **env_kwargs)

Stock Dimension: 29, State Space: 494


In [9]:
df_account_value, df_actions = CustomDRLAgent.DRL_prediction(
    model=learned_algorithms[len(learned_algorithms) - 1].learned_algorithm, environment=env_gym
)
# df_account_value = df_account_value.set_index('date')

hit end!


In [15]:
df_account_value

,date,account_value
0,2019-04-12,1.000000e+06
1,2019-04-15,9.994271e+05
2,2019-04-16,1.000304e+06
3,2019-04-17,9.962704e+05
4,2019-04-18,9.988517e+05
...,...,...
923,2022-12-09,1.241070e+06
924,2022-12-12,1.260773e+06
925,2022-12-13,1.278515e+06
926,2022-12-14,1.262068e+06


In [24]:
ax = sns.lineplot(data=df_account_value, x="date", y="account_value")
ax.plot()

AttributeError: 'UTC' object has no attribute 'update'

In [17]:
sns.lmplot(data=df_account_value, x="date", y="account_value")

ValueError: could not convert string to float: '2019-04-12'

In [18]:
perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv(program.exp_dir.out.results.joinpath("perf_stats_all_ppo_" + now_time() + ".csv"))

Annual return          0.057717
Cumulative returns     0.229539
Annual volatility      0.295385
Sharpe ratio           0.339129
Calmar ratio           0.130704
Stability              0.362444
Max drawdown          -0.441589
Omega ratio            1.068750
Sortino ratio          0.469728
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.037811
Daily value at risk   -0.036818
dtype: float64


In [19]:
# Baseline
baseline_df = get_baseline(ticker="^DJI", start=program.dataset["date"].min(), end=program.dataset["date"].max())

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (927, 8)
Annual return          0.070772
Cumulative returns     0.286005
Annual volatility      0.232728
Sharpe ratio           0.411239
Calmar ratio           0.190831
Stability              0.561379
Max drawdown          -0.370862
Omega ratio            1.087791
Sortino ratio          0.567932
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.953450
Daily value at risk   -0.028941
dtype: float64


In [20]:
print(baseline_df["date"].min())
print(baseline_df["date"].max())

2019-04-12
2022-12-14


In [21]:
stats = backtest_stats(baseline_df, value_col_name="close")

Annual return          0.070772
Cumulative returns     0.286005
Annual volatility      0.232728
Sharpe ratio           0.411239
Calmar ratio           0.190831
Stability              0.561379
Max drawdown          -0.370862
Omega ratio            1.087791
Sortino ratio          0.567932
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.953450
Daily value at risk   -0.028941
dtype: float64


In [22]:
backtest_plot(
    df_account_value,
    baseline_ticker="^DJI",
    baseline_start=program.dataset["date"].min(),
    baseline_end=program.dataset["date"].max(),
)

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (927, 8)


Start date,2019-04-12
End date,2022-12-15
Total months,44
,Backtest
Annual return,5.772%
Cumulative returns,22.954%
Annual volatility,29.539%
Sharpe ratio,0.34
Calmar ratio,0.13
Stability,0.36
Max drawdown,-44.159%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,44.16,2020-02-12,2020-03-20,2021-01-06,236
1,32.13,2021-11-02,2022-09-27,NaT,NaN
2,9.42,2019-07-23,2019-08-14,2019-09-11,37
3,7.11,2019-04-30,2019-06-03,2019-06-20,38
4,6.81,2021-08-27,2021-09-21,2021-10-18,37


Stress Events,mean,min,max
New Normal,0.04%,-15.71%,13.63%


In [27]:
import yfinance as yf
import pyfolio as pf

In [33]:
fb = yf.Ticker("META")
history = fb.history("max")
history

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2012-05-18 00:00:00-04:00,42.049999,45.000000,38.000000,38.230000,573576400,0,0
2012-05-21 00:00:00-04:00,36.529999,36.660000,33.000000,34.029999,168192700,0,0
2012-05-22 00:00:00-04:00,32.610001,33.590000,30.940001,31.000000,101786600,0,0
2012-05-23 00:00:00-04:00,31.370001,32.500000,31.360001,32.000000,73600000,0,0
2012-05-24 00:00:00-04:00,32.950001,33.209999,31.770000,33.029999,50237200,0,0
...,...,...,...,...,...,...,...
2022-12-13 00:00:00-05:00,122.129997,123.300003,118.639999,120.150002,44701100,0,0
2022-12-14 00:00:00-05:00,119.389999,124.139999,119.389999,121.589996,36922000,0,0
2022-12-15 00:00:00-05:00,118.330002,118.629997,114.010002,116.150002,34531000,0,0


In [34]:
history.index = history.index.tz_local

history.info()

returns = history.Close.pct_change()

pf.create_returns_tear_sheet(returns, live_start_date="2020-1-1")

TypeError: Already tz-aware, use tz_convert to convert.